<a href="https://colab.research.google.com/github/robsonglima/StockMarket_B3/blob/main/Analise_de_Correla%C3%A7%C3%A3o_Mercado_de_A%C3%A7%C3%B5es_B3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Premissas:

Determinados movimentos no mercado de ações influenciam os preços de diversos ativos na mesma direção

Determinados ativos têm maior correlação de preços do que outros. Exemplos ativos de commodities têm maior correlação do que uma ação de um banco comparado ao preço de uma ação de uma locadora de automóveis.

Outro exemplo: ações de bancos podem apresentar correlações significativas entre si. Assim como empresas de seguros entre si, ou como empresas de telecom e assim por diante.

Objetivo do projeto:

Identificar as ações de maior correlação, ou principais grupos de ações que têm maior correlação entre seus preços.
Descrever o comportamento dessas correlações tentando identificar em que períodos de tempo elas se manifestam de forma previsível (ao longo de minutos, horas, dias etc)
Escrever estratégias que possam ser testadas como teses de investimento com base no estudo de movimentos passados


In [ ]:
!pip install yfinance
!pip install plotly

In [ ]:
import pandas as pd
import yfinance as yf
import time
import io
import requests


Baixar as listadas na B3 - Filtra por cash e volume acima de 10K

https://www.b3.com.br/pt_br/market-data-e-indices/servicos-de-dados/market-data/consultas/boletim-diario/dados-publicos-de-produtos-listados-e-de-balcao/



TradeInformationConsolidatedFile_YYYYMMDD_1.csv

In [ ]:
# Caminho para o arquivo CSV baixado - Conferir nome
caminho_arquivo = "https://github.com/robsonglima/StockMarket_B3/blob/5c7977ff8b2f087ce8232a937cc39855d4adbed9/TradeInformationConsolidatedFile_20250127_1.csv?raw=true"

# Baixar o conteúdo do arquivo CSV como texto
response = requests.get(caminho_arquivo)
response.raise_for_status()  # Verificar se a solicitação foi bem-sucedida

# Ler o CSV usando io.StringIO para evitar problemas com encoding
df = pd.read_csv(io.StringIO(response.text), delimiter=';', encoding='latin1', header=1, on_bad_lines='skip')



In [ ]:
df.head()

In [ ]:
#Incluindo .SA no TRacker para consultar no Yahoo Finance
df['TckrSymb'] = df['TckrSymb'] + '.SA'

In [ ]:
# Filtrar as linhas onde a coluna 'SgmtNm' contém 'ODD LOT'
df_filtrado_segmento = df[df['SgmtNm'].str.contains('CASH', na=False)]

In [ ]:
df_filtrado_segmento.info()

In [ ]:
df_filtrado_segmento.info()

In [ ]:
# Selecionar as 15 ações com maior Qnt de Trader (TradQty) - Para Teste
df_top_15 = df_filtrado_segmento.nlargest(15, 'TradQty')

# Exibir as 15 ações selecionadas
print(df_top_15[['TckrSymb', 'TradQty']])

In [ ]:
#BUscar info do detor (mercado da acao - baseado na premissa)
def preencher_industry(df):
    industries = []

    for ticker in df['TckrSymb']:
        try:

            info = yf.Ticker(ticker).info
            industry = info.get("industry", "N/A")
            industries.append(industry)
            print(f"Industry para {ticker}: {industry}")
        except Exception as e:
            print(f"Erro ao consultar industry para {ticker}: {e}")
            industries.append("Erro")

    # Nova coluna 'Industry' ao Df
    df['Industry'] = industries
    return df

# Atualizar o df_top_15 com a coluna 'Industry'
df_top_15_atualizado = preencher_industry(df_top_15)

# Exibir o DataFrame atualizado
print(df_top_15_atualizado)

#opcao de exportar para csv
#df_top_15_atualizado.to_csv("df_top_15_com_industry.csv", index=False, sep=";")


In [ ]:
df_top_15_atualizado

In [ ]:
tickers_top_15 = df_top_15_atualizado['TckrSymb'].tolist()

# Função para consultar preços intraday no Yahoo Finance
def consultar_precos_intradiarios_yf(tickers, intervalo="15m", periodo="1d"):
    precos = []  # Lista para armazenar os dados de preços

    for ticker in tickers:
        try:
            dados = yf.download(ticker, interval=intervalo, period=periodo, progress=False)
            dados = dados.reset_index()  # Reseta o índice
            dados.columns = ["datetime", "open", "high", "low", "close", "volume"]
            dados["symbol"] = ticker  # Adicionar o símbolo como uma coluna

            # Selecionar as colunas na ordem correta
            dados = dados[["datetime", "symbol", "volume", "open", "high", "low", "close"]]
            precos.append(dados)  # Adicionar os dados ajustados à lista
            print(f"Dados obtidos para {ticker}")
        except Exception as e:
            print(f"Erro ao processar {ticker}: {e}")

    # Combinar todos os DataFrames em um único DataFrame
    return pd.concat(precos, ignore_index=True) if precos else pd.DataFrame()

# Consultar preços intradiários para todos os tickers do df_top_15
df_precos_intradiarios = consultar_precos_intradiarios_yf(tickers_top_15)

# Exibir os dados consolidados
print(df_precos_intradiarios.head())

# Salvar os dados em CSV, se necessário
df_precos_intradiarios.to_csv("precos_intradiarios_top_15.csv", index=False)

Incluir setor - Yfinance

In [ ]:
df_precos_intradiarios.head()

In [ ]:
# Adicionar a coluna 'Industry' ao DataFrame
df_precos_intradiarios['Industry'] = df_top_15_atualizado['Industry']

# Exibir o DataFrame atualizado
df_precos_intradiarios.info()




In [ ]:
df_top_15_atualizado.head()